In [ ]:
!pip install earthengine-api
!pip install earthengine-api pandas
!pip install geopandas
!pip install gdal
!pip install geemap
!pip install mss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.9 MB/s eta 0:00:00


## DESCRIPTION
###About the dataset:
The Copernicus DEM is a Digital Surface Model (DSM) which represents the surface of the Earth including buildings, infrastructure and vegetation. This DEM is derived from an edited DSM named WorldDEM&trade, i.e. flattening of water bodies and consistent flow of rivers has been included. Editing of shore- and coastlines, special features such as airports and implausible terrain structures has also been applied.

The WorldDEM product is based on the radar satellite data acquired during the TanDEM-X Mission, which is funded by a Public Private Partnership between the German State, represented by the German Aerospace Centre (DLR) and Airbus Defence and Space. More details are available in the dataset [documentation](https://spacedata.copernicus.eu/documents/20123/121239/GEO1988-CopernicusDEM-SPE-002_ProductHandbook_I4.0.pdf).

Earth Engine asset has been ingested from the DGED files.

[See more details](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_DEM_GLO30)

###About what we do:
We ...

### Resolution
- **30 meters**


### Bands:
[See more details](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_DEM_GLO30#bands)

| Name | Min | Max | Description |
|------|-----|-----|-------------|
| **DEM** | - | - | Digital Surface Model |
| **EDM** | 0 | 13 | The Edit Data Mask indicates all DEM pixels that were modified during the terrain and hydro editing process. |
| **FLM** | 0 | 101 | The Filling Mask is created primarily during the terrain editing process. |
| **HEM** | - | - | The Height Error Mask represents the corresponding height error for each DEM pixel in the form of the standard deviation derived from the interferometric coherence and geometrical considerations. |
| **WBM** | 0 | 3 | The Water Body Mask shows all DEM pixels classified as water and edited according to the categories Ocean, Lake, or River. |

## Terms of Use

The GLO-30 dataset is available worldwide with a free license with the exception of two countries (Armenia and Azerbaijan). [License for Copernicus DEM.](https://docs.sentinel-hub.com/api/latest/static/files/data/dem/resources/license/License-COPDEM-30.pdf)

In [ ]:
# Xác thực tài khoản Earth Engine
import ee
ee.Authenticate()

In [ ]:
# Khởi tạo Earth Engine
ee.Initialize(project='ee-nguyendangkhoi9517')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Đọc tệp CSV
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/gfd.csv')


In [ ]:
import ee
import pandas as pd
import numpy as np
import os
import requests

# Initialize Earth Engine
ee.Initialize(project='ee-nguyendangkhoi9517')

# Load DataFrame from CSV starting from row 418
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/gfd.csv')

# Define square size (10 km² -> 10,000 meters)
square_size_m = 10000  # each side of the square

# Loop through each row in the DataFrame starting from row 418
for index, row in df.iterrows():
    center_lon = row['dfo_centroid_x']
    center_lat = row['dfo_centroid_y']
    area_km2 = row['gfd_area']  # Get area from column

    # Calculate the radius of the area of interest in meters
    radius = np.sqrt(area_km2) * 1000 / 2  # Radius in meters

    # Skip if radius is zero or very close to zero
    if radius <= 0:
        print(f"Skipping event {index} due to zero radius.")
        continue

    # Parse and format the event's start and end dates
    try:
        began_date_raw = row['dfo_began']
        ended_date_raw = row['dfo_ended']

        # Convert two-digit years if necessary
        if len(began_date_raw.split('/')[-1]) == 2:
            began_date_raw = began_date_raw[:-2] + '20' + began_date_raw[-2:]
        if len(ended_date_raw.split('/')[-1]) == 2:
            ended_date_raw = ended_date_raw[:-2] + '20' + ended_date_raw[-2:]

        began_date = pd.to_datetime(began_date_raw, format='%m/%d/%Y', errors='coerce')
        ended_date = pd.to_datetime(ended_date_raw, format='%m/%d/%Y', errors='coerce')

        if pd.isna(began_date) or pd.isna(ended_date):
            print(f"Invalid date format for event {index}. Skipping this event.")
            continue

        # Format dates to strings
        began_date_str = began_date.strftime('%Y-%m-%d')
        ended_date_str = ended_date.strftime('%Y-%m-%d')
    except Exception as e:
        print(f"Error parsing dates for event {index}: {e}")
        continue

    # Create the central point and circular area around it
    point = ee.Geometry.Point(center_lon, center_lat)
    circle = point.buffer(radius)

    # Define the number of squares along each axis to cover the circle
    num_squares = int(np.ceil(radius / square_size_m))

    # Generate grid squares around the central point
    squares = []
    for i in range(-num_squares, num_squares + 1):
        for j in range(-num_squares, num_squares + 1):
            # Calculate the bounds for each 10 km² square
            square = ee.Geometry.Rectangle([
                center_lon + i * square_size_m / 111320 - square_size_m / 111320 / 2,
                center_lat + j * square_size_m / 111320 - square_size_m / 111320 / 2,
                center_lon + i * square_size_m / 111320 + square_size_m / 111320 / 2,
                center_lat + j * square_size_m / 111320 + square_size_m / 111320 / 2
            ])
            # Only add squares that intersect with the circle
            if circle.intersects(square):
                squares.append(square)

    print(f"Number of squares for event {index}: {len(squares)}")

    # Get elevation data from Copernicus DEM GLO-30
    dem = ee.ImageCollection('COPERNICUS/DEM/GLO30').select('DEM').mosaic()

    # Loop through each square and download elevation images
    for square_index, square in enumerate(squares):
        # Clip DEM to the square region
        dem_square = dem.clip(square)

        # Calculate the center of the square for filename purposes
        square_center = square.centroid().coordinates().getInfo()
        square_center_lat = square_center[1]
        square_center_lon = square_center[0]

        # Define visualization parameters for elevation
        elevationVis = {
            'min': 0.0,
            'max': 300.0,  # Adjust based on elevation range of your area
            'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
        }

        # Visualize the clipped DEM data
        dem_visualized = dem_square.visualize(**elevationVis)

        # Generate thumbnail URL for the elevation map
        thumbnail_params = {
            'region': square,
            'dimensions': 512,
            'format': 'png'
        }
        url = dem_visualized.getThumbURL(thumbnail_params)

        # Define the file path with the event date range included
        save_path = f"/content/drive/MyDrive/Colab Notebooks/elevation/elevation_map_{began_date_str}_{ended_date_str}_{square_center_lat}_{square_center_lon}.png"
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        # Download the image and save it to the specified path
        response = requests.get(url)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"Image for square {square_index} of event {index} saved to: {save_path}")
        else:
            print(f"Failed to download image for square {square_index} of event {index}")


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Image for square 34 of event 262 saved to: /content/drive/MyDrive/Colab Notebooks/elevation/elevation_map_2005-07-01_2005-08-12_44.83217948669057_125.41071188250028.png
Image for square 35 of event 262 saved to: /content/drive/MyDrive/Colab Notebooks/elevation/elevation_map_2005-07-01_2005-08-12_44.92201056765863_125.41071188249909.png
Image for square 36 of event 262 saved to: /content/drive/MyDrive/Colab Notebooks/elevation/elevation_map_2005-07-01_2005-08-12_44.20336191431274_125.50054299999907.png
Image for square 37 of event 262 saved to: /content/drive/MyDrive/Colab Notebooks/elevation/elevation_map_2005-07-01_2005-08-12_44.29319299666542_125.50054300000038.png
Image for square 38 of event 262 saved to: /content/drive/MyDrive/Colab Notebooks/elevation/elevation_map_2005-07-01_2005-08-12_44.383024078839945_125.50054299999788.png
Image for square 39 of event 262 saved to: /content/drive/MyDrive/Colab Notebooks/elevation/eleva